In [24]:

from config.database import client, collection_airports, collection_weather, collection_searchTrack, collection_gates       #Luis mongoDB.
from config.database import collection_flights, client_UJ, db_UJ        # UJ mongoDB
from routes.root.root_class import Root_class, Fetching_Mechanism, Source_links_and_api, Root_source_links
from bson import ObjectId
import requests
import datetime as dt
import json
import pickle
from pymongo import UpdateOne

# db = client_UJ.cirrostrats                 # get a particular db
cts = db_UJ['test_st']   # create/get a collection
collection = collection_flights
collection_st = collection_searchTrack
# collection_backup = client_UJ['cirrostrats']['flights_backup']
# collection = collection_searchTrack


In [34]:
    
pipeline = [
    {"$limit": 10000},
    # Unwind the versions array to examine each version individually
    # {"$unwind": "$versions"},
    
    # Filter to keep only versions that have a clearance field
    {"$match": 
        # {"versions.clearance": {"$exists": True}}},
    
    {'flightID': {'$regex': "^UCA"}}},
    # Project the specific fields you need
    {"$project": {'flightID':1, '_id':0}}
        # "c": "$versions"  # Rename the versions object to clearance_data for clarity
    # }},
    
    # Optional: If you only want one result per flight, you can group by flightID
    # {"$group": {
    #     "_id": "$flightID",
    #     "clearance_data": {"$first": "$clearance_data"}
    # }}
]
# Execute the pipeline on your collection
# clearance_documents = list(collection_flights.aggregate(pipeline))


In [ ]:
# # Create a test collection. Takes about a few 10 seconds. 
# collection_name = 'test_airports'
# db = client.cirrostrats                 # get a particular db
# collection_test = db[collection_name]   # create/get a collection
# docs = [doc for doc in collection_airports.find({}).limit(1000)]
# collection_test.insert_many(docs)
# cta = collection_test

# # collection_es = collection_test
# # id = [i['flightID'] for i in collection_es.find({},{'flightID':1, '_id':0})]


In [ ]:
# Mongo 101 - guide

# list all db
# client.list_database_names()
# db = client['cirrostrats']            # create a db
# collection = db['name_of_collection'] # create/get a collection. if it doesn't exist then get one.

# db = client.cirrostrats               # get a particular db
# all_db = db.list_collection_names()   # get all collections of the db
# collection = db['flights']            # get a particular collection


# ________________________________________________________

# get all documents. Expensive operation.
# x = [i for i in collection.find({})]

# return the first document
# one = collection.find_one({})

# Count documents
# collection_backup.count_documents({})

# Delete all documents from the collection
# collection_backup.delete_many({})

# return the first document that have a `count` field using the $exists operator
# collection.find_one({'count':{"$exists":True}})

# returns just the flightID and excludes ObjectID fields. Very fast and efficient. first argument is the filter by and second is the projection. In this case, 
# filters all documents(first argument {}) and projection(second argument) returns just the flightID field and excludes the ObjectID.
# criteria = {'flightID': 1, '_id': 0}
# collection.find({}, criteria)

# Get backup flight numbers from collection_backup(inefficient since it scans complete documents instead of just the flightNumber field) and insert them into the the empty collection
# Avoid use since its inefficient.
# docs = [{'flightNumber':i['flightNumber']} for i in collection_backup.find({})]
# collection.insert_many(docs)

# skim throguh only the flightID field from the collection(efficient) documents instead of getting all the fields from document.
# docs = [{'flightID':doc['flightID']} 
        # for doc in collection_other.find({}, {'flightID': 1, '_id': 0})]
# collection.insert_many(docs)

# add flight numbers to the collection- fast operation. does not account for uniques and wont avoid duplicates.
# docs = [{'flightNumber':i} for i in flightNumbers]
# collection.insert_many(docs)

# insert a key field and value in a document using set operator:
# collection.update_one({'flightID':'GJS123'},{'$set':{'count':7.92193}})

# unset operator - remove a field from a document
# collection.update_one({'flightID':'GJS123'},{'$unset':{'count':''}})


# This will insert flightnumbers in bulk in the collection if it doesnt exist - slow operation since
# it checks if the flightnumber already exist. !! WHY NOT SKIM THROUGH JUST flightID field instead of complete documents?
# update_operations = []
# docs = list(collection.find({}, {'_id': 0}))  # Exclude `_id` to prevent conflicts
# for i in docs:
#     flightNumber = i['flightID']
#     update_operations.append(
#         UpdateOne({'flightID': flightNumber},
#                   {'$set': {'flightID': flightNumber}},
#                   upsert=True
#                   )
#     ),
# result = collection_backup.bulk_write(update_operations)

# match all documents. works like find. will fetch all that matches gjs in `flightNumber` fields
# [i for i in collection_flights.find({'flightNumber':{'$regex': 'GJS'}})]

# limit documents to first 300. Saves memory and intensive operations.
# a = collection.find().limit(300)

In [ ]:
# Advanced mongo operations - pipepline and aggregation.

# Typical use..
# pipeline = [
    # {"$match": {"count": {"$exists": True}}},        # filter documents that have a count field
    # {'$project': {'flightNumber': 1, '_id': 0}},
    # {'$out': 'collection_flights'}
# ]


# # collection_flights.find_one({})
# # MongoDB aggregation pipeline to find all documents with a 'clearance' field in any nested dictionary within the 'versions' array
# pipeline = [
#     {"$limit": 10000},
#     # Unwind the versions array to examine each version individually
#     {"$unwind": "$versions"},
    
#     # Filter to keep only versions that have a clearance field
#     {"$match": {"versions.clearance": {"$exists": True}}},
    
#     # Project the specific fields you need
#     {"$project": {
#         "c": "$versions"  # Rename the versions object to clearance_data for clarity
#     }},
    
#     # Optional: If you only want one result per flight, you can group by flightID
#     # {"$group": {
#     #     "_id": "$flightID",
#     #     "clearance_data": {"$first": "$clearance_data"}
#     # }}
# ]
# # Execute the pipeline on your collection
# clearance_documents = list(collection_flights.aggregate(pipeline))


# filter match documents according to the query and sort. the $or operator matches either `flightNumber` or `name`(for airport collection)
# collection_merge = [collection_airports, collection_flights] 
# for coll in collection_merge:
#     pipeline = [
#             {"$match": {"count": {"$exists": True}}},        # filter documents that have a count field
#             {"$match": {"$or": [
#                 {"flightNumber": {"$regex": query, "$options": "i"}},       # matches flightNumber field in flights collection
#                 {"name": {"$regex": query, "$options": "i"}}                # matches name field in airport collection
#             ]}},
#             {"$sort": {"count": -1}},               # sort by popularity - the count field contains popularity rating.
#         ]
#     coll.aggregate(pipeline)
#     suggestions.extend(coll.aggregate(pipeline))


# Only return flightID from the root doc and version_created_at fields from the versions subdocument.
#     '$project': {
#         'flightID': 1,
#         'version_timestamps': '$versions.version_created_at'

# Change name field to uppercase in collection, e.g 'name': 'New York', will change to 'name': 'NEW YORK'
    # '$project': {
    #   'name': { '$toUpper': "$name" },

# Group documents by field and sum their unique counts. e.g the 'route' field will be analysed for uniqueness and their counts.
    # $group: {
    #   _id: "$route",
    #   count: { $sum: 1 }



# collection.aggregate(pipeline)



In [21]:
# redis call
import redis
r = redis.Redis(host="localhost", port=6379, db=0, decode_responses=True)


In [ ]:
# redis call 2
# import json

# all_keys = [i for i in r.keys("*")]
# all_keys.remove('multipleEntries')

# db = [json.loads(r.get(i)) for i in a]

In [ ]:
# Part 1: forMDB 
import requests
xx = requests.get("http://3.15.228.76:8000/flights")

repo_flights = []
scheduled_flights = []
for i in xx.json()['flightNumbers']:
    # first three digits are GJS, after those digits are flight number
    # if len(i) == 7 and i[:3] == 'UAL' and i[3:].isnumeric():
    if i[:3] == 'UAL' and i[3:].isnumeric():
        # if i[3] == '4':
        scheduled_flights.append(int(i[3:]))
        # elif i[3] == '3':
            # repo_flights.append(int(i[3:]))

jmsFlightNumbers = xx.json()['flightNumbers']
# Saving these list of flight numbers into the collection flightNumbers. !! WONT ACCOUNT FOR DUPLICATES!
# collection_flightNumbers.insert_many(jmsFlightNumbers)

import pickle
with open('JMS_flight_numbers.pkl', 'wb') as f:
    pickle.dump(jmsFlightNumbers, f)

from routes.root.weather_parse import Weather_parse
from routes.root.dep_des import Pull_flight_info

tots = {}
flt_info = Pull_flight_info()
for i in scheduled_flights:
    rets = flt_info.fs_dep_arr_timezone_pull(i)
    tots[rets['origin_fs']] = tots.get(rets['origin_fs'],0) + 1
    tots[rets['destination_fs']] = tots.get(rets['destination_fs'],0) + 1
    
# Sorting the dictionary by values
tots = {k: v for k, v in sorted(tots.items(), key=lambda item: item[1], reverse=True)}

new_tots = {}
for icao,flightIDs in tots.items():
    sum_of_values = sum(tots.values())
    new_tots[icao] = int(flightIDs/sum_of_values*100)+1

forMDB = {'scheduled_flights': scheduled_flights, 'repo_flights': repo_flights, 'popularity_raw': tots, 'popularity_proportions': new_tots}

with open(r'forMDB.pkl', 'wb') as f:
    pickle.dump(forMDB, f)


In [ ]:
# Part 2 forMDB
# adds `count` as 1 to all gjs flights 
with open(r'forMDB.pkl', 'rb') as f:
    forMDB = pickle.load(f)
GJS_scheduled_flights = forMDB['scheduled_flights']
GJS_repo_flights = forMDB['repo_flights']
popularity_raw = forMDB['popularity_raw']
popularity_proportions = forMDB['popularity_proportions']

# 
operations = []
for i in GJS_scheduled_flights:
# for airport, count in popularity_proportions.items():
    flightNumber = "GJS"+str(i)
    operation = UpdateOne(
        {'flightNumber': flightNumber},
        {'$set': {'count': 1}},
        upsert=True
    )
    operations.append(operation)

# Perform bulk write
result = collection_flights.bulk_write(operations)
# popularity_proportions
# collection_flights.find_one()

In [86]:
gjs_reg = {'N503GJ',
 'N504GJ',
 'N506GJ',
 'N508GJ',
 'N511GJ',
 'N520GJ',
 'N521GJ',
 'N522GJ',
 'N523GJ',
 'N524GJ',
 'N526GJ',
 'N534GJ',
 'N535GJ',
 'N536GJ',
 'N537GJ',
 'N538GJ',
 'N539GJ',
 'N540GJ',
 'N541GJ',
 'N543GJ',
 'N544GJ',
 'N546GJ',
 'N548GJ',
 'N549GJ',
 'N551GJ',
 'N552GJ',
 'N554GJ',
 'N556GJ',
 'N557GJ',
 'N559GJ',
 'N561GJ',
 'N563GJ',
 'N564GJ',
 'N566GJ',
 'N569GJ',
 'N578GJ',
 'N579GJ',}